In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedKFold

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

from multiprocessing import Pool, cpu_count
from threading import Thread
from multiprocessing import Queue

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [23]:
class ThreadWithReturnValue(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None
    def run(self):
        #print(type(self._target))
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return

In [3]:
def apply_smote(df):
    df.reset_index(drop=True,inplace=True)
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [6]:
def load_both_data(project,metric):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    understand_df['Name'] = understand_df.Name.str.rsplit('.',1).str[1]
    
    commit_guru_file_level_path = 'data/commit_guru_file_level/' + project + '_file.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_file_level_df = commit_guru_file_level_df[commit_guru_file_level_df['file_name'].str.contains('.java')]
    commit_guru_file_level_df['Name'] = commit_guru_file_level_df.file_name.str.rsplit('/',1).str[1].str.split('.').str[0].str.replace('/','.')
    commit_guru_file_level_df = commit_guru_file_level_df.drop('file_name',axis = 1)
    
    
    df = understand_df.merge(commit_guru_file_level_df,how='left',on=['commit_hash','Name'])
    
    
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    df = df[cols]
    
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
#     df.dropna(inplace=True)
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    imp_mean = IterativeImputer(random_state=0)
    X = imp_mean.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    
    if metric == 'process':
        X = X[['la', 'ld', 'lt', 'age', 'ndev', 'nuc', 'ns', 'exp', 'sexp', 'rexp', 'nd']]
    elif metric == 'product':
        X = X.drop(['la', 'ld', 'lt', 'age', 'ndev', 'nuc', 'ns', 'exp', 'sexp', 'rexp', 'nd'],axis = 1)
    else:
        X = X
    return X,y

In [7]:
def run_self(project,metric):
    X,y = load_both_data(project,metric)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    if metric == 'process':
        loc = X_test['la'] + X_test['lt']
    elif metric == 'product':
        loc = X_test.CountLineCode
    else:
        loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = clf.feature_importances_
    print(len(importance))
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [16]:
def run_self_k(project,projects,metric):
    precision = []
    recall = []
    pf = []
    f1 = []
    g_score = []
    auc = []
    pci_20 = []
    ifa = []
    importance = []
    X,y = load_both_data(project,metric)
    skf = StratifiedKFold(n_splits=5)
    for train_index, test_index in skf.split(X, y):
        X_train = X.loc[train_index]
        y_train = y.loc[train_index]
        df_smote = pd.concat([X_train,y_train],axis = 1)
        df_smote = apply_smote(df_smote)
        y_train = df_smote.Bugs
        X_train = df_smote.drop('Bugs',axis = 1)
        clf = RandomForestClassifier()
        clf.fit(X_train,y_train)
        importance = clf.feature_importances_
        for _project in projects:
            X_test,y_test = load_both_data(_project,metric)
            if metric == 'process':
                loc = X_test['la'] + X_test['lt']
            elif metric == 'product':
                loc = X_test.CountLineCode
            else:
                loc = X_test['la'] + X_test['lt']
            predicted = clf.predict(X_test)
            abcd = metrices.measures(y_test,predicted,loc)
            pf.append(abcd.get_pf())
            recall.append(abcd.calculate_recall())
            precision.append(abcd.calculate_precision())
            f1.append(abcd.calculate_f1_score())
            g_score.append(abcd.get_g_score())
            pci_20.append(abcd.get_pci_20())
            ifa.append(abcd.get_ifa())
            try:
                auc.append(roc_auc_score(y_test, predicted))
            except:
                auc.append(0)
            print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [21]:
def run(projects,all_projects):
    precision_list = {}
    recall_list = {}
    pf_list = {}
    f1_list = {}
    g_list = {}
    auc_list = {}
    pci_20_list = {}
    ifa_list = {}
    featue_importance = {}
    for project in projects:
        try:
            if project == '.DS_Store':
                continue
            print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
            recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance = run_self_k(project,all_projects,'process')
            recall_list[project] = recall
            precision_list[project] = precision
            pf_list[project] = pf
            f1_list[project] = f1
            g_list[project] = g_score
            auc_list[project] = auc
            pci_20_list[project] = pci_20
            ifa_list[project] = ifa
            featue_importance[project] = importance
        except Exception as e:
            print(e)
            continue
    final_result = {}
    final_result['precision'] = precision_list
    final_result['recall'] = recall_list
    final_result['pf'] = pf_list
    final_result['f1'] = f1_list
    final_result['g'] = g_list
    final_result['auc'] = auc_list
    final_result['pci_20'] = pci_20_list
    final_result['ifa'] = ifa_list
    final_result['featue_importance'] = featue_importance
    return final_result

In [ ]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [25]:
threads = []
results = {}
results['precision'] = {}
results['recall'] = {}
results['pf'] = {}
results['f1'] = {}
results['g'] = {}
results['auc'] = {}
results['pci_20'] = {}
results['ifa'] = {}
cores = cpu_count()
split_projects = np.array_split(projects, cores)
for i in range(cores):
    print("starting thread ",i)
    t = ThreadWithReturnValue(target = run, args = [split_projects[i],projects])
    threads.append(t)
for th in threads:
    th.start()
for th in threads:
    response = th.join()
    print(response)
    results['precision'].update(response['precision'])
    results['recall'].update(response['recall'])
    results['pf'].update(response['pf'])
    results['f1'].update(response['f1'])
    results['g'].update(response['g'])
    results['auc'].update(response['auc'])
    results['pci_20'].update(response['pci_20'])
    results['ifa'].update(response['ifa'])
with open('results/Performance/process+product_cross_project.pkl', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

starting thread  0
starting thread  1
starting thread  2
starting thread  3
starting thread  4
starting thread  5
starting thread  6
starting thread  7
+++++++++++++++++   Droppy  +++++++++++++++++
+++++++++++++++++   RxActivityResult  +++++++++++++++++
+++++++++++++++++   smart-show  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.97      0.65      0.78        48
           1       0.66      0.97      0.79        34

    accuracy                           0.78        82
   macro avg       0.81      0.81      0.78        82
weighted avg       0.84      0.78      0.78        82

              precision    recall  f1-score   support

           0       1.00      0.23      0.37        48
           1       0.48      1.00      0.65        34

    accuracy                           0.55        82
   macro avg       0.74      0.61      0.51        82
weighted avg       0.78      0.55      0.49        82

              precision    recall  f1-scor

              precision    recall  f1-score   support

           0       0.97      0.78      0.87       243
           1       0.38      0.85      0.53        39

    accuracy                           0.79       282
   macro avg       0.68      0.81      0.70       282
weighted avg       0.89      0.79      0.82       282

              precision    recall  f1-score   support

           0       1.00      0.17      0.29        48
           1       0.46      1.00      0.63        34

    accuracy                           0.51        82
   macro avg       0.73      0.58      0.46        82
weighted avg       0.78      0.51      0.43        82

              precision    recall  f1-score   support

           0       0.95      0.38      0.54        48
           1       0.52      0.97      0.68        34

    accuracy                           0.62        82
   macro avg       0.74      0.67      0.61        82
weighted avg       0.77      0.62      0.60        82

              preci

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       243
           1       0.97      1.00      0.99        39

    accuracy                           1.00       282
   macro avg       0.99      1.00      0.99       282
weighted avg       1.00      1.00      1.00       282

              precision    recall  f1-score   support

           0       0.99      0.94      0.96       243
           1       0.71      0.92      0.80        39

    accuracy                           0.94       282
   macro avg       0.85      0.93      0.88       282
weighted avg       0.95      0.94      0.94       282

              precision    recall  f1-score   support

           0       1.00      0.29      0.45        48
           1       0.50      1.00      0.67        34

    accuracy                           0.59        82
   macro avg       0.75      0.65      0.56        82
weighted avg       0.79      0.59      0.54        82

              preci

              precision    recall  f1-score   support

           0       0.83      0.06      0.11       243
           1       0.14      0.92      0.24        39

    accuracy                           0.18       282
   macro avg       0.48      0.49      0.18       282
weighted avg       0.74      0.18      0.13       282

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       243
           1       0.87      1.00      0.93        39

    accuracy                           0.98       282
   macro avg       0.93      0.99      0.96       282
weighted avg       0.98      0.98      0.98       282

              precision    recall  f1-score   support

           0       0.97      0.85      0.91       243
           1       0.47      0.85      0.61        39

    accuracy                           0.85       282
   macro avg       0.72      0.85      0.76       282
weighted avg       0.90      0.85      0.86       282

              preci

              precision    recall  f1-score   support

           0       0.89      0.98      0.93        41
           1       0.97      0.85      0.90        33

    accuracy                           0.92        74
   macro avg       0.93      0.91      0.92        74
weighted avg       0.92      0.92      0.92        74

              precision    recall  f1-score   support

           0       0.80      0.03      0.06       243
           1       0.14      0.95      0.24        39

    accuracy                           0.16       282
   macro avg       0.47      0.49      0.15       282
weighted avg       0.71      0.16      0.09       282

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       243
           1       1.00      1.00      1.00        39

    accuracy                           1.00       282
   macro avg       1.00      1.00      1.00       282
weighted avg       1.00      1.00      1.00       282

              preci

              precision    recall  f1-score   support

           0       0.75      0.73      0.74        41
           1       0.68      0.70      0.69        33

    accuracy                           0.72        74
   macro avg       0.71      0.71      0.71        74
weighted avg       0.72      0.72      0.72        74

              precision    recall  f1-score   support

           0       0.79      1.00      0.88        41
           1       1.00      0.67      0.80        33

    accuracy                           0.85        74
   macro avg       0.89      0.83      0.84        74
weighted avg       0.88      0.85      0.85        74

              precision    recall  f1-score   support

           0       0.95      0.58      0.72       243
           1       0.23      0.79      0.36        39

    accuracy                           0.61       282
   macro avg       0.59      0.69      0.54       282
weighted avg       0.85      0.61      0.67       282

              preci

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        41
           1       1.00      0.97      0.98        33

    accuracy                           0.99        74
   macro avg       0.99      0.98      0.99        74
weighted avg       0.99      0.99      0.99        74

              precision    recall  f1-score   support

           0       0.76      0.78      0.77        41
           1       0.72      0.70      0.71        33

    accuracy                           0.74        74
   macro avg       0.74      0.74      0.74        74
weighted avg       0.74      0.74      0.74        74

              precision    recall  f1-score   support

           0       0.91      0.98      0.94        41
           1       0.97      0.88      0.92        33

    accuracy                           0.93        74
   macro avg       0.94      0.93      0.93        74
weighted avg       0.93      0.93      0.93        74

              preci

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        48
           1       1.00      0.82      0.90        34

    accuracy                           0.93        82
   macro avg       0.94      0.91      0.92        82
weighted avg       0.93      0.93      0.93        82

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       1.00      1.00      1.00        33

    accuracy                           1.00        74
   macro avg       1.00      1.00      1.00        74
weighted avg       1.00      1.00      1.00        74

              precision    recall  f1-score   support

           0       0.70      0.76      0.73        41
           1       0.67      0.61      0.63        33

    accuracy                           0.69        74
   macro avg       0.69      0.68      0.68        74
weighted avg       0.69      0.69      0.69        74

              preci

              precision    recall  f1-score   support

           0       1.00      0.46      0.63        48
           1       0.57      1.00      0.72        34

    accuracy                           0.68        82
   macro avg       0.78      0.73      0.68        82
weighted avg       0.82      0.68      0.67        82

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        48
           1       0.92      1.00      0.96        34

    accuracy                           0.96        82
   macro avg       0.96      0.97      0.96        82
weighted avg       0.97      0.96      0.96        82

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       1.00      1.00      1.00        33

    accuracy                           1.00        74
   macro avg       1.00      1.00      1.00        74
weighted avg       1.00      1.00      1.00        74

              preci